# Demonstration of the Entmoot API

This notebook includes comparisons of the Entmoot strategy to other Bofire 
strategies.

## Defining model parameters

Model parameters to be passed into `EntingStrategy` can be defined in a few ways. 
Throughout this notebook, they are defined using a dictionary of parameters. 
However, all of the below are valid. 
The use of `EntingParams` is encouraged, however it does require more imports to use.

In [1]:
import entmoot.models.model_params as ent_params

# all of these are valid to pass into `EntingStrategy(enting_params=params)`

# as a dictionary
params0 = {"unc_params": {"dist_metric": "l1", "acq_sense": "exploration"}}
# populate EntingParams with dictionary unpacking
params1 = ent_params.EntingParams(**params0)
# use objects directly
params2 = ent_params.EntingParams(
    unc_params=ent_params.UncParams(
        dist_metric="l1",
        acq_sense="exploration"
    )
)
# explictly create all dataclass instances
params3 = ent_params.EntingParams(
    unc_params=ent_params.UncParams(
        dist_metric="l1",
        acq_sense="exploration"
    ),
    tree_train_params=ent_params.TreeTrainParams(
        train_params=ent_params.TrainParams()
    )
)

params1 == params2 == params3

True

## Single objective Bayesian Optimization strategy

This section includes a comparison to the Sobo strategy given in the "Getting 
Started" docs. The API is identical, with additional options given to the 
data model. Note that the EntingStrategy only supports one candidate, as 
each generated candidate is optimal (so generating multiple would generate 
duplicates).

In [2]:
from bofire.benchmarks.single import Himmelblau
import bofire.strategies.mapper as strategy_mapper

benchmark = Himmelblau()

samples = benchmark.domain.inputs.sample(10)
experiments = benchmark.f(samples, return_complete=True)

C:\Users\tobyb\phd\bofire\bofire\surrogates\xgb.py:12: UserWarning: xgboost not installed, BoFire's `XGBoostSurrogate` cannot be used.
  warnings.warn("xgboost not installed, BoFire's `XGBoostSurrogate` cannot be used.")


In [3]:
from bofire.data_models.strategies.api import SoboStrategy
from bofire.data_models.acquisition_functions.api import qNEI

sobo_strategy_data_model = SoboStrategy(domain=benchmark.domain, acquisition_function=qNEI())
sobo_strategy = strategy_mapper.map(sobo_strategy_data_model)

sobo_strategy.tell(experiments=experiments)
sobo_strategy.ask(candidate_count=2)


,x_1,x_2,y_pred,y_sd,y_des
0,5.028809,0.282956,158.004321,314.289086,-158.004321
1,-0.299493,0.858126,75.802201,276.831445,-75.802201


In [4]:
from bofire.data_models.strategies.api import EntingStrategy

enting_params = {"unc_params": {"dist_metric": "l1", "acq_sense": "exploration"}}
solver_params = {"solver_name": "gurobi"}

enting_strategy_data_model = EntingStrategy(
    domain=benchmark.domain, solver_params=solver_params,
    enting_params=enting_params,                                        
    learn_from_candidates_coeff=10.0
)

enting_strategy = strategy_mapper.map(enting_strategy_data_model)

enting_strategy.tell(experiments=experiments)
enting_strategy.ask(candidate_count=2)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-06
Read LP format model from file C:\Users\tobyb\AppData\Local\Temp\tmp3me86j93.pyomo.lp
Reading time = 0.00 seconds
x1: 1121 rows, 648 columns, 3774 nonzeros
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1121 rows, 648 columns and 3774 nonzeros
Model fingerprint: 0x0273e6a9
Variable types: 602 continuous, 46 integer (46 binary)
Coefficient statistics:
  Matrix range     [6e-04, 4e+02]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 6e+00]
  RHS range        [6e-02, 6e+00]
Presolve removed 151 rows and 92 columns
Presolve time: 0.00s
Presolved: 970 rows, 556 columns, 2916 nonzeros
Variable types: 530 continuous, 26 integer (26 binary)
Found heuristic solution: objective 193.7483728

,x_1,x_2,y_pred,y_sd,y_des
0,6.000000,-3.349438,1.807020,0.386341,-1.807020
0,5.446058,0.000000,2.165424,0.403288,-2.165424


When `candidate_count > 1`, the strategy will fit to each candidate in order to produce the next candidate. This means that sequential calls to `EntingStrategy.ask()` will tend to be further from the ground truth.

In [5]:
enting_strategy.ask(candidate_count=5)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-06
Read LP format model from file C:\Users\tobyb\AppData\Local\Temp\tmpxyg_225v.pyomo.lp
Reading time = 0.02 seconds
x1: 1375 rows, 782 columns, 4705 nonzeros
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1375 rows, 782 columns and 4705 nonzeros
Model fingerprint: 0xb7bf3842
Variable types: 728 continuous, 54 integer (54 binary)
Coefficient statistics:
  Matrix range     [7e-03, 4e+02]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 6e+00]
  RHS range        [6e-02, 6e+00]
Presolve removed 49 rows and 49 columns
Presolve time: 0.00s
Presolved: 1326 rows, 733 columns, 4191 nonzeros
Variable types: 704 continuous, 29 integer (29 binary)
Found heuristic solution: objective 165.8761717

,x_1,x_2,y_pred,y_sd,y_des
0,3.643090,-0.160579,212.354874,0.286032,-212.354874
0,6.000000,6.000000,1.976358,0.472637,-1.976358
0,5.723030,-2.215150,5.962139,0.242492,-5.962139
0,5.723030,2.999999,-15.058899,0.369514,15.058899
0,3.654947,3.601530,-42.435354,0.333518,42.435354


## Multi Objective BO

In [6]:
from bofire.benchmarks.multi import DTLZ2
import bofire.strategies.mapper as strategy_mapper

benchmark = DTLZ2(dim=2, num_objectives=2)

samples = benchmark.domain.inputs.sample(10)
experiments = benchmark.f(samples, return_complete=True)

In [7]:
enting_params = {"unc_params": {"dist_metric": "l1", "acq_sense": "exploration"}}
solver_params = {"solver_name": "gurobi"}

enting_strategy_data_model = EntingStrategy(
    domain=benchmark.domain, solver_params=solver_params,
    enting_params=enting_params,                                        
    learn_from_candidates_coeff=10.0
)
enting_strategy = strategy_mapper.map(enting_strategy_data_model)

enting_strategy.tell(experiments=experiments)
enting_strategy.ask(candidate_count=1)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-06
Read LP format model from file C:\Users\tobyb\AppData\Local\Temp\tmpc3tulzyp.pyomo.lp
Reading time = 0.01 seconds
x1: 2536 rows, 1406 columns, 8825 nonzeros
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2536 rows, 1406 columns and 8825 nonzeros
Model fingerprint: 0x40eeaac1
Variable types: 1360 continuous, 46 integer (46 binary)
Coefficient statistics:
  Matrix range     [7e-08, 1e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-02, 1e+00]
Presolve removed 194 rows and 109 columns
Presolve time: 0.02s
Presolved: 2342 rows, 1297 columns, 8641 nonzeros
Variable types: 1271 continuous, 26 integer (26 binary)
Found heuristic solution: objective 0.14

,x_0,x_1,f_0_pred,f_1_pred,f_0_sd,f_1_sd,f_0_des,f_1_des
0,1.0,0.0,0.239611,0.999267,0.491768,0.491768,-0.239611,-0.999267


## Maximize single objective

In [8]:
from bofire.benchmarks.single import Ackley

benchmark = Ackley()

samples = benchmark.domain.inputs.sample(10)
experiments = benchmark.f(samples, return_complete=True)

In [9]:
enting_params = {"unc_params": {"dist_metric": "l1", "acq_sense": "exploration"}}
solver_params = {"solver_name": "gurobi"}

enting_strategy_data_model = EntingStrategy(
    domain=benchmark.domain, solver_params=solver_params,
    enting_params=enting_params,                                        
    learn_from_candidates_coeff=10.0
)
enting_strategy = strategy_mapper.map(enting_strategy_data_model)

enting_strategy.tell(experiments=experiments)
enting_strategy.ask(candidate_count=1)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-06
Read LP format model from file C:\Users\tobyb\AppData\Local\Temp\tmpoq8unk2t.pyomo.lp
Reading time = 0.01 seconds
x1: 1085 rows, 630 columns, 3596 nonzeros
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1085 rows, 630 columns and 3596 nonzeros
Model fingerprint: 0x639898d2
Variable types: 584 continuous, 46 integer (46 binary)
Coefficient statistics:
  Matrix range     [1e-07, 6e+01]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 3e+01]
  RHS range        [2e-02, 3e+01]
Presolve removed 75 rows and 49 columns
Presolve time: 0.00s
Presolved: 1010 rows, 581 columns, 2992 nonzeros
Variable types: 555 continuous, 26 integer (26 binary)
Found heuristic solution: objective -17.9471755

,x_1,x_2,y_pred,y_sd,y_des
0,-32.768,-27.639076,19.890957,0.370347,19.890957


## Compare domain to problem config

BoFire defines its problems using Domains, whereas ENTMOOT uses ProblemConfigs. 
This demonstrates the conversion from Domain to ProblemConfig, using the multi 
objective categorical problem defined in the ENTMOOT benchmark.

In [10]:
from bofire.data_models.domain.api import Domain, Inputs, Outputs
from bofire.data_models.features.api import (
    CategoricalInput,
    ContinuousInput,
    DiscreteInput,
    ContinuousOutput,
)
from bofire.data_models.objectives.api import MinimizeObjective


def build_multi_obj_categorical_problem(n_obj: int = 2, no_cat=False) -> Domain:
    """
    Builds a small test example which is used in Entmoot tests.
    """

    cat_feat = (
        []
        if no_cat
        else [CategoricalInput(key="x0", categories=("blue", "orange", "gray"))]
    )
    input_features = Inputs(
        features=cat_feat
        + [
            DiscreteInput(key="x1", values=[5, 6]),
            DiscreteInput(key="x2", values=[0, 1]),  # binary
            ContinuousInput(key="x3", bounds=[5.0, 6.0]),
            ContinuousInput(key="x4", bounds=[4.6, 6.0]),
            ContinuousInput(key="x5", bounds=[5.0, 6.0]),
        ]
    )

    output_features = Outputs(
        features=[
            ContinuousOutput(
                key=f"y{i}", objective=MinimizeObjective(w=1.0, bounds=[0.0, 1.0])
            )
            for i in range(n_obj)
        ]
    )

    domain = Domain(inputs=input_features, outputs=output_features)

    return domain


In [11]:
from bofire.utils.entmoot import domain_to_problem_config
import entmoot.benchmarks as ent
from entmoot.problem_config import ProblemConfig

domain = build_multi_obj_categorical_problem()
problem_config, _ = domain_to_problem_config(domain)
problem_config_ent = ProblemConfig()
ent.build_multi_obj_categorical_problem(problem_config_ent)

print(problem_config, "\n", problem_config_ent)


PROBLEM SUMMARY
---------------
features:
x0 :: Categorical :: ('blue', 'orange', 'gray') 
x1 :: Integer :: (5, 6) 
x2 :: Binary :: (0, 1) 
x3 :: Real :: (5.0, 6.0) 
x4 :: Real :: (4.6, 6.0) 
x5 :: Real :: (5.0, 6.0) 

objectives:
y0 :: MinObjective
y1 :: MinObjective 
 
PROBLEM SUMMARY
---------------
features:
feat_0 :: Categorical :: ('blue', 'orange', 'gray') 
feat_1 :: Integer :: (5, 6) 
feat_2 :: Binary :: (0, 1) 
feat_3 :: Real :: (5.0, 6.0) 
feat_4 :: Real :: (4.6, 6.0) 
feat_5 :: Real :: (5.0, 6.0) 

objectives:
obj_0 :: MinObjective
obj_1 :: MinObjective
